In [1]:
# IMPORTS
##########################

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import os
import json

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

Using TensorFlow backend.


In [ ]:
# ONE RUN
#############################################

param = { 
    
    "batch_size" : 10, # 30,50,60,[70]
    "target_update_freq" : 1000,#10000], # 10000,20000,30000,[40000]
    "gamma" : 0.99,# # 0.98,0.99,0.995,[0.999]
    "train_freq" : 1, # 2,3,4,[5]
    "max_size" : 100000,#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : 3000, # 1000,2000,3000,[4000]
    "policy" : "epsGreedy",#, "epsGreedy"],
    "eps" : 0.1,#, 0.3, 0.1],
    "delta_time" : 10,
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

sumo_RL = simulation.simulator(
                    connection_label = "single_worker",
                    q_network_type = 'linear',
                    target_q_network_type = 'linear',
                    gamma = param["gamma"],
                    target_update_freq = param["target_update_freq"],
                    train_freq = param["train_freq"],
                    num_burn_in = 200,
                    batch_size = param["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = param["max_ep_length"],
                    experiment_id = "T",
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,19),
                    num_actions = 2,
                    use_gui = False,
                    delta_time = param["delta_time"],

                   # memory parameters
                    max_size = param["max_size"],

                   # additional parameters

                    policy = param["policy"],
                    eps = param["eps"],
                    num_episodes = 20,
                    monitoring = True,
                    episode_recording = True,
                    hparams = param.keys())

sumo_RL.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")

# agent.ddqn.train(env = agent.env, num_episodes = 100, policy = agent.policy, connection_label = agent.connection_label)

#evaluation_results = agent.evaluate(runs=2, use_gui= False)
#evaluation_results
# data= agent.ddqn.evaluate(env = agent.env, policy = "greedy")

In [ ]:
np.array([(np.array([2,2])+np.array([1,1])),[3,3]])

In [ ]:
sumo_RL.load("./Logs/2_state_test/run_7/model_checkpoints/run2_state_test_iter0.h5")

In [ ]:
sumo_RL.evaluate(5, use_gui = True)

In [ ]:
sumo_RL.ddqn.q_network.layers[0].get_weights()

In [ ]:
sumo_RL.load("./Logs/2_state_test/run_10/model_checkpoints/run2_state_test_iter0.h5")

In [ ]:
sumo_RL.ddqn.q_network.layers[0].get_weights()

In [ ]:
sumo_RL.load("./Logs/2_state_test/run_16/model_checkpoints/run2_state_test_iter0.h5")

In [ ]:
sumo_RL.ddqn.q_network.layers[0].get_weights()

In [ ]:
sumo_RL.load("./Logs/2_state_test/run_16/model_checkpoints/run2_state_test_iter22000.h5")

In [ ]:
sumo_RL.ddqn.q_network.layers[0].get_weights()

In [ ]:
sumo_RL.load("./Logs/2_state_test/run_10/model_checkpoints/run2_state_test_iter33000.h5")

In [ ]:
sumo_RL.ddqn.q_network.layers[0].get_weights()

In [ ]:
# GRIDSEARCH
###############################################################

experiment_id = "Linear fixed low eps" 


param = { 
    
    "batch_size" : [50], # 30,50,60,[70]
    "target_update_freq" : [1000],  #10000], # 10000,20000,30000,[40000]
    "gamma" : [0.99],# # 0.98,0.99,0.995,[0.999]
    "train_freq" : [3], # 2,3,4,[5]
    "max_size" : [100000],#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : [2000], # 1000,2000,3000,[4000]
    "policy" : ["epsGreedy","epsGreedy","epsGreedy","epsGreedy"],#, "epsGreedy"],
    "eps" : [0.05],#, 0.3, 0.1],
    "delta_time" : [10],
    "reward" : ["balanced","negative"]
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

param_grid = iter_params(**param)



def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all obje
    
    # print('Run', position + 1, '-- parameters', args)
    
    sumo_RL = simulation.simulator(
                    connection_label = position + 1,
                    q_network_type = 'linear',
                    target_q_network_type = 'linear',
                    gamma = args["gamma"],
                    target_update_freq = args["target_update_freq"],
                    train_freq = args["train_freq"],
                    num_burn_in = 10000,
                    batch_size = args["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = args["max_ep_length"],
                    experiment_id = experiment_id,
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    net_file = "cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,19),
                    num_actions = 2,
                    use_gui = False,
                    delta_time = args["delta_time"],
                    reward = args["reward"],

                   # memory parameters
                    max_size = args["max_size"],

                   # additional parameters

                    policy = args["policy"],
                    eps = args["eps"],
                    num_episodes = 300,
                    monitoring = True,
                    episode_recording = False,
                    hparams = args.keys())
    
    # print("training agent", position + 1)
    sumo_RL.train()
    # print("evaluating agent", position + 1)
    evaluation_results = sumo_RL.evaluate(runs = 5)

    return (position + 1, args, evaluation_results, evaluation_results["average_delay"])


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
        
#     with open(os.path.join("./logs",experiment_id,"GS_results"), "w",newline='') as file:
#             writer = csv.writer(file, dialect = 'excel')
#             writer.writerow(experiment_id)                
      
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
            json.dump(results[-1], file , indent=4)
                  
        #print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
    scores = [result[-1] for result in results]
    lowest = min(scores)
    winner = results[scores.index(lowest)]
    return winner, results

multiprocessing.freeze_support()
winner, results = gridsearch(param_grid)

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
...done filling replay memory
Run 7 -- running episode 1 / 300
...done filling replay memory
Run 5 -- running episode 1 / 300
...done filling replay memory
Run 4 -- running episode 1 / 300
...done filling replay memory
Run 1 -- running episode 1 / 300
...done filling replay memory
Run 2 -- running episode 1 / 300
...done filling replay memory
Run 3 -- running episode 1 / 300
...done filling replay memory
Run 8 -- running episode 1 / 300
...done filling replay memory
Run 6 -- running episode 1 / 300
Run 7 -- running episode 2 / 300
Run 5 -- running episode 2 / 300
Run 4 -- running episode 2 / 300
Run 1 -- running episode 2 / 300
Run 2 -- running episode 2 / 300
Run 3 -- running episode 2 / 300
Run 8 -- running episode 2 / 300
Run 6 -- running episode 2 / 300
Run 7 -- running episode 3 / 300
Run 5 -- running episode 3 / 300
Run 4 

Run 3 -- running episode 29 / 300
Run 7 -- running episode 30 / 300
Run 4 -- running episode 31 / 300
Run 2 -- running episode 31 / 300
Run 5 -- running episode 30 / 300
Run 8 -- running episode 31 / 300
Run 6 -- running episode 31 / 300
Run 3 -- running episode 30 / 300
Run 7 -- running episode 31 / 300
Run 4 -- running episode 32 / 300
Run 2 -- running episode 32 / 300
Run 5 -- running episode 31 / 300
Run 8 -- running episode 32 / 300
Run 6 -- running episode 32 / 300
Run 3 -- running episode 31 / 300
Run 7 -- running episode 32 / 300
Run 4 -- running episode 33 / 300
Run 2 -- running episode 33 / 300
Run 8 -- running episode 33 / 300
Run 5 -- running episode 32 / 300
Run 6 -- running episode 33 / 300
Run 4 -- running episode 34 / 300
Run 7 -- running episode 33 / 300
Run 2 -- running episode 34 / 300
Run 3 -- running episode 32 / 300
Run 8 -- running episode 34 / 300
Run 5 -- running episode 33 / 300
Run 6 -- running episode 34 / 300
Run 1 -- running episode 27 / 300
Run 4 -- runni

Run 8 -- running episode 69 / 300
Run 7 -- running episode 63 / 300
Run 6 -- running episode 69 / 300
Run 5 -- running episode 63 / 300
Run 3 -- running episode 64 / 300
Run 8 -- running episode 70 / 300
Run 6 -- running episode 70 / 300
Run 7 -- running episode 64 / 300
Run 5 -- running episode 64 / 300
Run 3 -- running episode 65 / 300
Run 8 -- running episode 71 / 300
Run 6 -- running episode 71 / 300
Run 4 -- running episode 67 / 300
Run 7 -- running episode 65 / 300
Run 3 -- running episode 66 / 300
Run 5 -- running episode 65 / 300
Run 6 -- running episode 72 / 300
Run 7 -- running episode 66 / 300
Run 3 -- running episode 67 / 300
Run 6 -- running episode 73 / 300
Run 5 -- running episode 66 / 300
Run 8 -- running episode 72 / 300
Run 7 -- running episode 67 / 300
Run 3 -- running episode 68 / 300
Run 6 -- running episode 74 / 300
Run 5 -- running episode 67 / 300
Run 2 -- running episode 56 / 300
Run 7 -- running episode 68 / 300
Run 6 -- running episode 75 / 300
Run 3 -- runni

Run 7 -- running episode 120 / 300
Run 3 -- running episode 120 / 300
Run 6 -- running episode 138 / 300
Run 3 -- running episode 121 / 300
Run 7 -- running episode 121 / 300
Run 2 -- running episode 64 / 300
Run 6 -- running episode 139 / 300
Run 3 -- running episode 122 / 300
Run 6 -- running episode 140 / 300
Run 4 -- running episode 79 / 300
Run 7 -- running episode 122 / 300
Run 6 -- running episode 141 / 300
Run 3 -- running episode 123 / 300
Run 8 -- running episode 88 / 300
Run 7 -- running episode 123 / 300
Run 6 -- running episode 142 / 300
Run 3 -- running episode 124 / 300
Run 8 -- running episode 89 / 300
Run 7 -- running episode 124 / 300
Run 6 -- running episode 143 / 300
Run 8 -- running episode 90 / 300
Run 1 -- running episode 35 / 300
Run 3 -- running episode 125 / 300
Run 6 -- running episode 144 / 300
Run 7 -- running episode 125 / 300
Run 8 -- running episode 91 / 300
Run 2 -- running episode 65 / 300
Run 6 -- running episode 145 / 300
Run 3 -- running episode 126

Run 3 -- running episode 171 / 300
Run 6 -- running episode 174 / 300
Run 4 -- running episode 92 / 300
Run 8 -- running episode 120 / 300
Run 3 -- running episode 172 / 300
Run 7 -- running episode 193 / 300
Run 7 -- running episode 194 / 300
Run 2 -- running episode 81 / 300
Run 7 -- running episode 195 / 300
Run 6 -- running episode 175 / 300
Run 7 -- running episode 196 / 300
Run 1 -- running episode 42 / 300
Run 7 -- running episode 197 / 300
Run 3 -- running episode 173 / 300
Run 7 -- running episode 198 / 300
Run 2 -- running episode 82 / 300
Run 8 -- running episode 121 / 300
Run 5 -- running episode 83 / 300
Run 7 -- running episode 199 / 300
Run 7 -- running episode 200 / 300
Run 6 -- running episode 176 / 300
Run 7 -- running episode 201 / 300
Run 6 -- running episode 177 / 300
Run 7 -- running episode 202 / 300
Run 2 -- running episode 83 / 300
Run 3 -- running episode 174 / 300
Run 7 -- running episode 203 / 300
Run 4 -- running episode 93 / 300
Run 3 -- running episode 17

In [ ]:
winner

In [ ]:
results

In [ ]:
decay_test = []
eps_test = 0.8
for i in range(300000):
    eps_test *= omega_test ** i
    decay_test.append(eps_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(300000),decay_test)

In [ ]:
first_results = pd.DataFrame(results).sort_values(by = 3)

In [ ]:
a = np.array([[1],[1]])
b = np.array([[1],[1]])

In [ ]:
np.array([[np.sum(nextstate[:,15]+ nextstate[:,17])],[np.sum(nextstate[:,16]+nextstate[:,18])]])

In [ ]:
a.shape

In [ ]:
np.sum(a[:,0]+b[:,0])

In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Flatten

input_shape = (2,)
model = Sequential()
    #model.add(Flatten(input_shape=input_shape, name="Layer_1")) # If a vector the flattening does not have any effect. (only matrices)
    #model.add(Dense(num_actions, input_shape=input_shape, activation=None, use_bias= True, name = "Hidden"))
model.add(Dense(3, input_shape=input_shape, activation=None, use_bias= True, name = "Output"))

In [ ]:
model.layers[0].get_weights()